# OpenVirus Jupyter Notebook

### AMIDICT:  getpapers + dictionary creation 

NOTE: _All work is done in a single folder, hence paths to specific files needn't be specified_

Ambreen Hamadani


#### GETPAPERS 
"This downloads articles from EPMC; it may take some minutes if there are hundreds of files, especially large PDFs. Ignore the warnings about "Not Open Access", but you will end up with maybe 5% fewer papers than you asked for "
#### Using Get papers to create a communal copus" 


#### Installation of getpapers

Requirements: NodeJS should be installed on your PC

In [1]:
!npm install --global getpapers

C:\Program Files\nodejs\getpapers -> C:\Program Files\nodejs\node_modules\getpapers\bin\getpapers.js
C:\Program Files\nodejs
`-- getpapers@0.4.17 
  +-- chalk@1.0.0 
  | +-- ansi-styles@2.2.1 
  | +-- escape-string-regexp@1.0.5 
  | +-- has-ansi@1.0.3 
  | | +-- ansi-regex@1.1.1 
  | | `-- get-stdin@4.0.1 
  | +-- strip-ansi@2.0.1 
  | `-- supports-color@1.3.1 
  +-- commander@2.7.1 
  | `-- graceful-readlink@1.0.1 
  +-- crossref@0.1.2 
  | +-- got@5.1.0 
  | | +-- create-error-class@2.0.1 
  | | | `-- capture-stack-trace@1.0.1 
  | | +-- is-plain-obj@1.1.0 
  | | +-- is-redirect@1.0.0 
  | | +-- node-status-codes@1.0.0 
  | | +-- object-assign@4.1.1 
  | | +-- parse-json@2.2.0 
  | | | `-- error-ex@1.3.2 
  | | |   `-- is-arrayish@0.2.1 
  | | +-- pinkie-promise@1.0.0 
  | | | `-- pinkie@1.0.0 
  | | +-- read-all-stream@3.1.0 
  | | | `-- pinkie-promise@2.0.1 
  | | |   `-- pinkie@2.0.4 
  | | +-- unzip-response@1.0.2 
  | | `-- url-parse-lax@1.0.0 
  | `-- request@2.65.0 
  |   +-- 

npm WARN deprecated request@2.65.0: request has been deprecated, see https://github.com/request/request/issues/3142
npm WARN deprecated node-uuid@1.4.8: Use uuid module instead
npm WARN deprecated tough-cookie@2.2.2: ReDoS vulnerability parsing Set-Cookie https://nodesecurity.io/advisories/130
npm WARN deprecated hawk@3.1.3: This module moved to @hapi/hawk. Please make sure to switch over as this distribution is no longer supported and may contain bugs and critical security issues.
npm WARN deprecated har-validator@2.0.6: this library is no longer supported
npm WARN deprecated sntp@1.0.9: This module moved to @hapi/sntp. Please make sure to switch over as this distribution is no longer supported and may contain bugs and critical security issues.
npm WARN deprecated boom@2.10.1: This version has been deprecated in accordance with the hapi support policy (hapi.im/support). Please upgrade to the latest version to get the best features, bug fixes, and security patches. If you are unable to

In [8]:
##setting the variables for the getpapers query 
HITS= 95
QUERY= "viral epidemic"
OUTDIR = "v_epid"
LOGFILE= OUTDIR+ "/"+ "log.txt"
FORMATS="-x -p"

To understand the functionality of the getpapers tool run the following command

In [5]:
!getpapers --help


  Usage: getpapers [options]

  Options:

    -h, --help                output usage information
    -V, --version             output the version number
    -q, --query <query>       search query (required)
    -o, --outdir <path>       output directory (required - will be created if not found)
    --api <name>              API to search [eupmc, crossref, ieee, arxiv] (default: eupmc)
    -x, --xml                 download fulltext XMLs if available
    -p, --pdf                 download fulltext PDFs if available
    -s, --supp                download supplementary files if available
    -t, --minedterms          download text-mined terms if available
    -l, --loglevel <level>    amount of information to log (silent, verbose, info*, data, warn, error, or debug)
    -a, --all                 search all papers, not just open access
    -n, --noexecute           report how many results match the query, but don't actually download anything
    -f, --logfile <filename>  save log to speci

In [6]:
!getpapers -q "$QUERY" -o $OUTDIR -f $LOGFILE  $FORMATS -k $HITS

info: Saving logs to ./v_epid/log.txtÂ 
info: Searching using eupmc API
info: Found 47963 open access results
warn: This version of getpapers wasn't built with this version of the EuPMC api in mind
warn: getpapers EuPMCVersion: 5.3.2 vs. 6.4 reported by api
info: Limiting to 95 hits

info: Done collecting results
info: limiting hits
info: Saving result metadata
info: Full EUPMC result metadata written to eupmc_results.json
info: Individual EUPMC result metadata records written
info: Extracting fulltext HTML URL list (may not be available for all articles)
info: Fulltext HTML URL list written to eupmc_fulltext_html_urls.txt
warn: Article with doi "10.1101/2020.05.11.20098087 did not have a PMCID (therefore no XML)
warn: Article with doi "10.1101/2020.04.02.20048892 did not have a PMCID (therefore no XML)
warn: Article with pmcid "PMC7194687" was not Open Access (therefore no XML)
warn: Article with pmcid "PMC7571379" was not Open Access (therefore no XML)
warn: Article with pmcid "PMC74

#### Dictionaries are made using WIKIDATA SPARQL 
The following code is about building the country dictionary first using SPARQL and then using amidict to validate 

In [1]:
#!pip install SPARQLWrapper 
from SPARQLWrapper import SPARQLWrapper, JSON,  XML
import pandas as pd

In [7]:
sparql = SPARQLWrapper("https://query.wikidata.org/sparql", agent="Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11")


In [14]:
sparql.setQuery("""
## This query contains phrases common to all queries and others which are specific to particular dictionaries.(SPECIFIC) 
## Select Query was used to retrieve specific results (Country name, wiki data number, Synonyms) 

## ?code as ?_iso3166 is optional and is specific to countries 

SELECT ?wikidata ?wikidataLabel ?wikipedia (?wikidataAltLabel as ?alt) ?synonym (?wikidataLabel as ?term) ?wikidataDescription
## (SPECIFIC) links to ?wikidata below
    (?code as ?_iso3166) ?coords { 

## Forcing particular query execution order
  hint:Query hint:optimizer "None" . 

## all ISO countries (SPECIFIC) must link to ?code above. 
  ?wikidata wdt:P297 ?code.
  ?wikidata wdt:P625 ?coords.

## Optional details about the terms like links to wikipaedia pages for each wikipedia page to be presented in a seperate column
  OPTIONAL { ?wikipedia schema:about ?wikidata; schema:isPartOf <https://en.wikipedia.org/> }
  SERVICE wikibase:label {
    bd:serviceParam wikibase:language "en".

## Selecting the prefered label 
    ?wikidata skos:altLabel ?wikidataAltLabel ; rdfs:label ?wikidataLabel; schema:description  ?wikidataDescription          
  } 

## (SPECIFIC) Making sure the RGI alphabets of the flags are not rendered as flags and they appear as simple alphabets by specifying the acceptable characters. 
  BIND (REPLACE(REPLACE(?wikidataAltLabel, "(, )?[🇦-🇿]{2}", ""), "^, ", "") AS ?synonym )
      }
""")

sparql.setReturnFormat(XML)
results = sparql.query().convert()

sparql.setReturnFormat(JSON)
results1 = sparql.query().convert()

print("THE SPARL QUERY RESULT LOOKS LIKE THIS: ")
print("*******************************************")

results_df = pd.io.json.json_normalize(results1['results']['bindings'])
results_df[['wikidata.value', 'wikidataLabel.value', '_iso3166.value', 'wikipedia.value', 'alt.value', 'synonym.value', 'coords.value' , 'term.value']].head()


THE SPARL QUERY RESULT LOOKS LIKE THIS: 
*******************************************


,wikidata.value,wikidataLabel.value,_iso3166.value,wikipedia.value,alt.value,synonym.value,coords.value,term.value
0,http://www.wikidata.org/entity/Q16,Canada,CA,https://en.wikipedia.org/wiki/Canada,"CA, ca, CDN, can, CAN, British North America, ...","CA, ca, CDN, can, CAN, British North America, ...",Point(-109.0 56.0),Canada
1,http://www.wikidata.org/entity/Q55,Netherlands,NL,https://en.wikipedia.org/wiki/Netherlands,"NL, Holland, Nederland, NED, nl, the Netherlan...","NL, Holland, Nederland, NED, nl, the Netherlan...",Point(5.55 52.316666666),Netherlands
2,http://www.wikidata.org/entity/Q96,Mexico,MX,https://en.wikipedia.org/wiki/Mexico,"MEX, Mexican Republic, MX, mx, 🇲🇽, United Mexi...","MEX, Mexican Republic, MX, mx, United Mexican ...",Point(-102.0 23.0),Mexico
3,http://www.wikidata.org/entity/Q262,Algeria,DZ,https://en.wikipedia.org/wiki/Algeria,"ALG, dz, 🇩🇿, People’s Democratic Republic of A...","ALG, dz, People’s Democratic Republic of Algeria",Point(1.0 28.0),Algeria
4,http://www.wikidata.org/entity/Q79,Egypt,EG,https://en.wikipedia.org/wiki/Egypt,"EGY, eg, Republic of Egypt, Arab Rep. Egypt, A...","EGY, eg, Republic of Egypt, Arab Rep. Egypt, A...",Point(29.0 27.0),Egypt


Save the sparql endpoint in country.xml

In [13]:
with open("dictionary\country.sparql.xml", "w", encoding="utf-8") as f:
    f.write(results.toxml())

### Installing ami
Requirements: JDK, Maven, git

In [ ]:
!git clone https://github.com/petermr/ami3.git
!cd ami3
!mvn install -Dmaven.test.skip=true

##### The Results should look like this 

#### AMIDICT
amidict converts the SPARQL output into the required dictionary format and also to validate it

In [16]:
!amidict --help

Usage: amidict [OPTIONS] COMMAND

`amidict` is a command suite for managing dictionary:

Parameters:
      [@<filename>...]   One or more argument files containing options.
Options:
  -d, --dictionary=<dictionaryNameList>[,<dictionaryNameList>...]...
                         input or output dictionary NAMES/s. for 'create' must be singular; when 'display' or
                           'translate', any number. Names should be lowercase, unique. [a-z][a-z0-9._]. Dots can be
                           used to structure dictionaries intodirectories. Dictionary names are relative to
                           'directory'. If <directory> is absent then dictionary names are absolute. ) This doesn't
                           make sense; it should relate to current working directory.
      --directory=<directory>
                         top directory containing dictionary/s. Subdirectories will use structured names (NYI). Thus
                           dictionary 'animals' is found in '<dire

In [ ]:
!amidict -vv --dictionary country --directory dictionary  --input dictionary/country.sparql.xml create --informat wikisparqlxml --sparqlmap name=wikidataLabel,term=wikidataLabel,description=wikidataDescription,wikidataURL=wikidata,wikidataID=wikidata,wikipediaPage=wikipedia,wikipediaURL=wikipedia,_p297_country=_iso3166,_coords=coords --transformName wikidataID=EXTRACT(wikidataURL,.*/(.*)) --synonyms=synonym

##### The Results should look like this 

The dictionary is then validated using amidict

In [ ]:
!amidict --dictionary country --directory dictionary display --validate

##### The results should look like this: